# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770863021576                   -0.52    9.0         
  2   -2.772147338971       -2.89       -1.33    1.0    143ms
  3   -2.772170201375       -4.64       -2.42    1.0    174ms
  4   -2.772170642166       -6.36       -3.12    1.0    148ms
  5   -2.772170722837       -7.09       -4.30    2.0    169ms
  6   -2.772170722967       -9.89       -4.63    1.0    194ms
  7   -2.772170723013      -10.33       -5.63    1.0    154ms
  8   -2.772170723015      -11.74       -6.01    2.0    180ms
  9   -2.772170723015      -13.12       -7.30    1.0    173ms
 10   -2.772170723015      -14.51       -7.74    2.0    201ms
 11   -2.772170723015      -15.05       -8.32    2.0    194ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770743371188                   -0.52    9.0         
  2   -2.7

1.7735578737999809

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770546511064                   -0.53    8.0         
  2   -2.772048913283       -2.82       -1.28    1.0    146ms
  3   -2.772082187988       -4.48       -2.88    1.0    220ms
  4   -2.772083398984       -5.92       -3.44    2.0    169ms
  5   -2.772083413632       -7.83       -3.73    2.0    170ms
  6   -2.772083417768       -8.38       -5.26    1.0    185ms
  7   -2.772083417810      -10.37       -5.69    2.0    451ms
  8   -2.772083417810   +  -13.07       -5.65    2.0    242ms
  9   -2.772083417811      -12.45       -6.56    1.0    664ms
 10   -2.772083417811      -14.03       -7.17    1.0    175ms
 11   -2.772083417811   +  -14.12       -7.91    2.0    187ms
 12   -2.772083417811   +  -15.35       -8.25    1.0    182ms

Polarizability via ForwardDiff:       1.772534984692124
Polarizability via finite difference: 1.7735578737999809
